In [1]:
import pandas as pd

df = pd.read_csv("../../../data/wsdm_mini.csv")
df.head()

,title1_zh,title2_zh,label
0,千叶湖八岁孩子不想去学英语，跳楼了,「辟谣」千叶湖八岁孩子跳楼了为谣言信息,disagreed
1,喝酸奶真的能补充益生菌吗？,喝酸奶来补充益生菌，靠谱么？,agreed
2,刚刚马云终于出手了！房价要跌，扬言房地产中介都要失业了,最新消息马云终于出手了！扬言房地产中介都要失业！6,agreed
3,直击“冯乡长”李正春追悼会：赵本山全程操办，赵四刘能现场祭奠,昆明会议直击“活摘”谣言,unrelated
4,李雨桐爆薛之谦离婚内幕，说到底就是网红之间的恩怨情仇嘛,薛之谦前女友李雨桐再次发微博爆料，薛之谦工作室发声明辟谣,disagreed


In [2]:
# 合并文本，然后对文本进行分词处理 

df['title_zh'] = df[['title1_zh', 'title2_zh']].apply(lambda x: ''.join(x), axis=1)  # 合并文本数据列
df_merge = df.drop(df.columns[[0, 1]], axis=1)  # 删除原文本列
df_merge.head()

,label,title_zh
0,disagreed,千叶湖八岁孩子不想去学英语，跳楼了「辟谣」千叶湖八岁孩子跳楼了为谣言信息
1,agreed,喝酸奶真的能补充益生菌吗？喝酸奶来补充益生菌，靠谱么？
2,agreed,刚刚马云终于出手了！房价要跌，扬言房地产中介都要失业了最新消息马云终于出手了！扬言房地产中介...
3,unrelated,直击“冯乡长”李正春追悼会：赵本山全程操办，赵四刘能现场祭奠昆明会议直击“活摘”谣言
4,disagreed,李雨桐爆薛之谦离婚内幕，说到底就是网红之间的恩怨情仇嘛薛之谦前女友李雨桐再次发微博爆料，薛之...


In [3]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip('\n') for line in f.readlines()]
    return stopwords
stopwords = load_stopwords('../../../data/stopwords.txt')
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '\u3000',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',


In [5]:
from tqdm.notebook import tqdm
import jieba

# 利用 jieba 进行分词，并且取出停用词

corpus = []
for line in tqdm(df['title_zh']):
    words = []
    seg_list = list(jieba.cut(line))  # 分词
    for word in seg_list:
        if word in stopwords:  # 删除停用词
            continue
        words.append(word)
    corpus.append(words)

  0%|          | 0/15000 [00:00<?, ?it/s]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\91658\AppData\Local\Temp\jieba.cache
Loading model cost 0.457 seconds.
Prefix dict has been built successfully.


In [8]:
import numpy as np
from gensim.models import Word2Vec

model = Word2Vec(corpus)  # 词嵌入，默认 size=100

# 整个句子所有单词向量的和，得到空间向量上这个句子的向量
def sum_vec(text):
    vec = np.zeros(100).reshape((1, 100))  # 初始化一个和 Word2Vec 嵌入等长度的 0 向量
    for word in text:
        # 得到句子中每个词的词向量并累加在一起
        if word in list(model.wv.index_to_key):
            vec += model.wv.get_vector(word).reshape((1, 100))
        else:
            pass
    return vec

# 将词向量保存为 Ndarray
X = np.concatenate([sum_vec(z) for z in tqdm(corpus)])
X.shape

  0%|          | 0/15000 [00:00<?, ?it/s]

(15000, 100)

In [19]:
import tensorflow as tf

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer

In [20]:
tokenizer.fit_on_texts(corpus)
X_ = tokenizer.texts_to_sequences(corpus)

for seq in X_[:1]:
    print([tokenizer.index_word[idx] for idx in seq])

X = tf.keras.preprocessing.sequence.pad_sequences(X_, maxlen=20)
X.shape

['千叶', '湖', '八岁', '孩子', '不想', '英语', '跳楼', '「', '辟谣', '千叶', '湖', '八岁', '孩子', '跳楼', '谣言', '信息']


(15000, 20)

In [22]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_onehot = encoder.fit_transform(df.label.values.reshape(len(df), -1))
y_onehot

<15000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 15000 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(10000, 16, input_length=20))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, 64, 10, validation_data=(X_test, y_test))

Epoch 1/10


TypeError: Failed to convert elements of SparseTensor(indices=Tensor("data_1:0", shape=(None, 2), dtype=int64), values=Tensor("data_2:0", shape=(None,), dtype=float32), dense_shape=Tensor("data_3:0", shape=(2,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.